In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scipy
import warnings
import math
warnings.filterwarnings('ignore')

use **jupyter nbconvert --to script FeatureExtraction.ipynb**
to convert notebook to script

Feature Extraction

In [2]:
# extract one feature for each different part of one step
def extractFeatures(partDicts):
    #list of lists (each with 1 timestamp) of all features of 1 step
    allFeatures=[]
    times=[]
    for part in partDicts:
        #print(part)
        features=[]
        # for 1 timeframe
        i=0
       # print("length of dicts:")
       # print(len(part))
        #print("frame: {}".format(part))
        for fr in part:
            
            try:
                #print("append feature")
                #print(i)
                # divide left  by right side
                if i==0:
                    #print("append time: {} : ".format(fr))
                    times.append(fr)
                    # !!!!!!!!!!!!!!!!!!!!!!!! 10= anzahl features
                #elif i==5:#10:
                    #print("last one")
                    #print("append last feature to : {} : {}".format(fr, fr.iloc[:,0:3].median().median()/ fr.iloc[:,3:6].median().median()))
                #    features.append(fr.iloc[:,0:3].median().median()/ fr.iloc[:,3:6].median().median())
                else:
                    #print("normal")
                    #print("append feature to : {} : {}".format(fr, fr.median().median()))
                    features.append(fr.median().median())
                #print(features)
                i+=1
            except:
                times.append(fr)
                #print("time:")
                #print(fr)
                i+=1
        allFeatures.append(features)
        #print("features {}".format(features))
        #print(features)
    #print(allFeatures)
    # print(times)
    li=[]
    li.append(times)
    #print("times : {}".format(times))
    li.append(allFeatures)
    #print("features : {}".format(features))
    return li 

Merge the features of 1 step by considering the 8 step phases, create new features and return a final list of features

In [3]:
# use the list of features to create a list of Features for 8 phases (8 matrices per step)
def createFeatureListForSteps(listOfFeatures):

    # separate one step into phases to get similar amount of features (based on https://d-nb.info/972323945/34 )
    # Phase 1 : 0-2% 
    # Phase 2 : 2-10%
    # Phase 3 : 10-30%  
    # Phase 4 : 30-50% 
    # Phase 5 : 50-60%
    # Phase 6 : 60-73%
    # Phase 7 : 73-87% 
    # Phase 8 : 87-100%  

    # create new list with one list for each step with each 8 lists (for each phase) with each 6 features (for each part of the insole) 
    stepFeatures=[]
    print(listOfFeatures)
    for li in listOfFeatures:
        try:
            # create new list with specific number of features 
            listOfPhasedFeatures=[]
            # 100% = total time for 1 step
            timestamps= li[0]
            features= li[1]
            print("features{}".format(li))
            print("timestamps {}".format(timestamps))
        
            
            start= li[0][0]
            end= li[0][-1]
            total= end- start
            
            numberOfMatrices= len(timestamps)
            # divide the data into percentages:
            phases=[]
            # 1. calc 1%:
            onePerc= total/100
            #print("1 % : {}".format(onePerc))
            # 2. calc the time for the ending of each phase
            phase1 = start + 2* onePerc
            phases.append(phase1)
            phase2 = phase1 + 8* onePerc
            phases.append(phase2)
            phase3 = phase2 + 20* onePerc
            phases.append(phase3)
            phase4 = phase3 + 20* onePerc
            phases.append(phase4)
            phase5 = phase4 + 10* onePerc
            phases.append(phase5)
            phase6 = phase5 + 13* onePerc
            phases.append(phase6)
            phase7 = phase6 + 14* onePerc
            phases.append(phase7)
            phase8 = phase7 + 13* onePerc
            phases.append(phase8)
            
            # determine nearest timestamps for end of phase
            timeIndex = 0 
            phaseInd=[]
            for phase in phases:
                # find indices for new phase
                while (timestamps[timeIndex] < phase) & (abs(phase - timestamps[timeIndex])> 15) :
                    timeIndex +=1
                phaseInd.append(timeIndex)
           
            
            if math.isinf(total):
                print("mistake in phases, infinite timestamp")
            else:
                # combine the 6 features of each matrix of 1 phase 
                featureIndex=0
                phaseFeatures=[]
                listOf1PhaseFeatures=[]
                for index in phaseInd:
                    #print("index : {}".format(index))
                    while (index!=featureIndex):
                        listOf1PhaseFeatures.append(features[featureIndex])
                    #   print("featureIndex : {}".format(featureIndex))
                        featureIndex+=1
                    # append the last one when index == featureIndex
                    listOf1PhaseFeatures.append(features[featureIndex])
                    #listOf1PhaseFeatures.append(features[len(features)-1])
                    # calculate the medium of each value in the list 
                    #print("listOf1PhaseFeatures: {}".format(listOf1PhaseFeatures))
                    listOfPhasedFeatures.append([sum(x)/len(listOf1PhaseFeatures) for x in zip(*listOf1PhaseFeatures)])   
                    listOf1PhaseFeatures= []
                #print("listOfPhasedFeatures: {}".format(listOfPhasedFeatures))
                # append extra feature (left div by right)
                #listOfPhasedFeatures.append(features[len(features)-1])
                stepFeatures.append(listOfPhasedFeatures)
                #stepFeatures.append(features[len(features)-1])
                #print(stepFeatures)
                featureIndex=0
        except:
            print("problem in extracting features")
            
    return stepFeatures

In [4]:
# merge the features of different steps to one list
# function to merge a specific number of steps to one single step
def mergeSteps(featureList, numfeatures, numberSteps): 

    remainingSteps= int(len(featureList)/numberSteps)
    
    newList=[]
    for i in range(remainingSteps):
        newList.append([[0] * numfeatures, [0] * numfeatures, [0] * numfeatures, [0] * numfeatures, [0] * numfeatures, [0] * numfeatures, [0] * numfeatures, [0] * numfeatures])
   
    countSteps=-1
    j=0
    lenCount=0
    # if there are too little steps available
    if len(featureList) < numberSteps:
        print("Error for merging the steps in Feature Extraction. Choose a smaller number of steps, as not enough steps are in the Feature List")
        return []
    
    for liste in featureList:
        countSteps+=1
        
        # if you have enough steps merged then start a new batch
        if(countSteps==numberSteps):
            
            j+=1 
            countSteps=0
            
        i=0
        for teil in liste:
           
            newList[j][i]= [sum(element) for element in zip(newList[j][i],teil)] 
                
            i+=1
        # if there are not enough steps to continue, then stop here
        lenCount+=1
        
        if countSteps==True:
            if (len(featureList)-lenCount < numberSteps):
                
                break
        
        
    for k in range(len(newList)):
        
        for l in range(len(newList[k])):
            
            newList[k][l]= [element/numberSteps for element in newList[k][l]]       
    
    return newList